<a href="https://colab.research.google.com/github/jeanmhuang/Daily-Quant-Notes/blob/main/9_14_2025_Daily_Quant_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Daily Quant Notes — 2025-09-14

A clean, repeatable daily workflow for market prep, review, and hypothesis tracking.

**How to use:**
1) Run the setup cell to install packages.
2) Set your watchlist & dates in the **Parameters** cell.
3) Run each section top-to-bottom. Edit the notes tables in place.
4) At the end, export a daily markdown summary and (optionally) save logs to Google Drive.



In [ ]:
# %%capture
# ✅ Setup: install/upgrade core libs (Colab-friendly)
!pip -q install --upgrade yfinance pandas pandas_ta plotly pandas_datareader matplotlib pandas_market_calendars


In [ ]:
# Parameters — edit me
from datetime import datetime, timedelta

# Market universe: add/remove tickers as needed
WATCHLIST = [
    "SPY","QQQ","IWM","DIA",
    "XLK","XLF","XLY","XLP","XLV","XLI","XLE","XLU","XLB","XLRE","XLC",
    "TLT","IEF","SHY","HYG","LQD",
    "^VIX","GC=F","CL=F","NG=F","DX-Y.NYB","BTC-USD","ETH-USD"
]

BENCHMARK = "SPY"
START_DAYS = 365  # lookback for charts & indicators
START = (datetime.today() - timedelta(days=START_DAYS)).strftime("%Y-%m-%d")
END = datetime.today().strftime("%Y-%m-%d")

# Optional: your custom idea tickers
IDEA_TICKERS = ["NVDA","AAPL","MSFT","META","AMZN"]

print(f"Universe size: {len(WATCHLIST)} | Start: {START} End: {END}")


In [ ]:
# Imports
import pandas as pd, numpy as np, yfinance as yf, pandas_ta as ta
import plotly.express as px
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import pandas_market_calendars as mcal
from datetime import datetime
import textwrap, os, pathlib

pd.set_option("display.float_format", lambda x: f"{x:,.4f}")


## 1) Market Calendar & Session Info

In [ ]:
# Trading calendar (NYSE) and today's session context
nyse = mcal.get_calendar('XNYS')
schedule = nyse.schedule(start_date=pd.Timestamp.today().normalize() - pd.Timedelta(days=14),
                         end_date=pd.Timestamp.today().normalize() + pd.Timedelta(days=7))

today = pd.Timestamp.today(tz='America/New_York')
is_trading_day = nyse.valid_days(today.normalize(), today.normalize()).size > 0
next_open = nyse.schedule(start_date=today, end_date=today + pd.Timedelta(days=7))['market_open'].iloc[0] if not is_trading_day else pd.NaT

print(f"Today (NY): {today:%Y-%m-%d %H:%M %Z}")
print(f"Is trading day? {is_trading_day}")
display(schedule.tail(10))

## 2) Price Data Download

In [ ]:
# Download adjusted prices for WATCHLIST
def fetch_prices(tickers, start, end):
    try:
        data = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)
        if isinstance(data.columns, pd.MultiIndex):
            data = data['Close']
        else:
            data = data.rename(columns={'Close':'Close'})
        return data.dropna(how='all')
    except Exception as e:
        print("Download error:", e)
        return pd.DataFrame()

prices = fetch_prices(WATCHLIST, START, END)
print(prices.shape)
display(prices.tail())

## 3) Snapshot — Returns & Heatmap

In [ ]:
# Compute return windows
def compute_returns(df):
    ret = pd.DataFrame(index=df.columns)
    last = df.ffill().iloc[-1]
    ret['1D'] = df.ffill().pct_change().iloc[-1]
    ret['5D'] = df.ffill().pct_change(5).iloc[-1]
    ret['1M'] = df.ffill().pct_change(21).iloc[-1]
    ret['3M'] = df.ffill().pct_change(63).iloc[-1]
    ret['6M'] = df.ffill().pct_change(126).iloc[-1]
    ret['YTD'] = df[df.index >= f"{datetime.today().year}-01-01"].ffill().pct_change().iloc[-1]
    ret['Price'] = last.values
    return ret.sort_values('1D', ascending=False)

returns = compute_returns(prices)
display(returns.style.background_gradient(cmap='RdYlGn', subset=['1D','5D','1M','3M','6M','YTD']).format("{:.2%}"))

# Plot benchmark vs a few assets
subset = prices[[c for c in prices.columns if c in [BENCHMARK,"^VIX","TLT","CL=F","DX-Y.NYB"] and c in prices.columns]]
if not subset.empty:
    norm = subset / subset.iloc[0] * 100
    norm.plot(figsize=(10,5), title="Indexed Performance (100 = start)")
    plt.xlabel("Date"); plt.ylabel("Indexed Level")
    plt.show()


## 4) Technicals — RSI & MACD (Idea Tickers)

In [ ]:
def add_technicals(ticker, start=START, end=END):
    df = fetch_prices([ticker], start, end)
    if df.empty:
        return pd.DataFrame()
    s = df[ticker].to_frame('Close')
    s['RSI14'] = ta.rsi(s['Close'], length=14)
    macd = ta.macd(s['Close'], fast=12, slow=26, signal=9)
    s = pd.concat([s, macd], axis=1)
    return s.dropna()

tech = {t: add_technicals(t) for t in IDEA_TICKERS}
# Show latest snapshot
latest_rows = []
for t, df in tech.items():
    if df.empty: continue
    latest = df.iloc[-1][['Close','RSI14','MACD_12_26_9','MACDs_12_26_9','MACDh_12_26_9']]
    latest.name = t
    latest_rows.append(latest)
if latest_rows:
    tech_df = pd.DataFrame(latest_rows)
    display(tech_df.style.format({'Close':'{:.2f}','RSI14':'{:.1f}','MACD_12_26_9':'{:.2f}','MACDs_12_26_9':'{:.2f}','MACDh_12_26_9':'{:.2f}'}))


## 5) Rates & Macro — FRED Yields (Optional)

In [ ]:
# Try to pull a few FRED series; skip gracefully if blocked
FRED_SERIES = {
    "DGS2":"2Y Treasury",
    "DGS10":"10Y Treasury",
    "T10Y2Y":"10Y-2Y Spread"
}
fred = {}
for code, name in FRED_SERIES.items():
    try:
        s = pdr.DataReader(code, 'fred', START, END).dropna()
        fred[name] = s[code]
    except Exception as e:
        print(f"FRED fetch failed for {name} ({code}):", e)

if fred:
    fred_df = pd.DataFrame(fred).dropna()
    display(fred_df.tail())
    fred_df.plot(figsize=(10,5), title="FRED Yields / Spreads")
    plt.xlabel("Date"); plt.ylabel("Percent / Spread")
    plt.show()


## 6) Volatility — VIX & Realized

In [ ]:
# VIX and simple realized vol (close-to-close) on SPY
def realized_vol(series, window=21):
    returns = series.pct_change()
    return returns.rolling(window).std() * np.sqrt(252)

out = {}
if "^VIX" in prices.columns:
    out['VIX'] = prices["^VIX"]
if BENCHMARK in prices.columns:
    out['RealizedVol_SPY_21d'] = realized_vol(prices[BENCHMARK])

vol_df = pd.DataFrame(out).dropna()
display(vol_df.tail())
vol_df.plot(figsize=(10,5), title="Vol Dashboard")
plt.xlabel("Date"); plt.ylabel("Level / Vol")
plt.show()


## 7) Ideas Tracker — Hypotheses & Triggers

In [ ]:
# Editable table for today’s hypotheses
ideas_cols = [
    "date","ticker","thesis","signal/setup","entry","stop","target","confidence_1to5","status"
]
try:
    ideas = pd.read_csv("ideas_log.csv")
    # Keep only known columns
    ideas = ideas[[c for c in ideas_cols if c in ideas.columns]]
except:
    ideas = pd.DataFrame(columns=ideas_cols)

today_str = pd.Timestamp.today().strftime("%Y-%m-%d")
new_row = {c:"" for c in ideas_cols}
new_row["date"] = today_str
ideas = pd.concat([ideas, pd.DataFrame([new_row])], ignore_index=True)
display(ideas.tail(10))

# Save anytime after editing the DF in-place:
# ideas.to_csv("ideas_log.csv", index=False)


## 8) P&L & Risk Log

In [ ]:
pnl_cols = ["date","strategy","gross_pnl","fees","net_pnl","notes"]
risk_cols = ["date","exposure_long","exposure_short","gross_exposure","net_exposure","var_estimate","notes"]

def init_or_load(path, cols):
    try:
        df = pd.read_csv(path)
        df = df[[c for c in cols if c in df.columns]]
    except:
        df = pd.DataFrame(columns=cols)
    return df

pnl = init_or_load("pnl_log.csv", pnl_cols)
risk = init_or_load("risk_log.csv", risk_cols)

for df, cols in [(pnl, pnl_cols),(risk, risk_cols)]:
    if df.empty or (pd.Timestamp.today().strftime("%Y-%m-%d") not in df.get('date',[]).values):
        df.loc[len(df.index)] = {c:"" for c in cols}
        df.at[len(df.index)-1, "date"] = pd.Timestamp.today().strftime("%Y-%m-%d")

print("P&L Log (tail):")
display(pnl.tail(5))
print("Risk Log (tail):")
display(risk.tail(5))

# Example save:
# pnl.to_csv("pnl_log.csv", index=False)
# risk.to_csv("risk_log.csv", index=False)


## 9) Notes — Morning Plan / Midday Check / EOD Review

In [ ]:
# Free-form notes with headers. Edit inline as needed.
TEMPLATE = f"""
### Morning Plan ({pd.Timestamp.today():%Y-%m-%d})
- Themes:
- Key levels:
- Data today:
- What would make me wrong?

### Midday Check
- What changed?
- New risks?
- Adds/trims?

### End-of-Day Review
- What worked / didn't?
- What did I learn?
- What to carry into tomorrow?
"""

try:
    with open("daily_notes.md","r") as f:
        existing = f.read()
except:
    existing = ""

if TEMPLATE.strip() not in existing:
    existing += "\n\n" + TEMPLATE

print(existing)
# Save after editing:
# with open("daily_notes.md","w") as f: f.write(existing)


## 10) Export Daily Summary (Markdown)

In [ ]:
# Compose a compact summary from current tables
from io import StringIO

def fmt_returns(df, names=("SPY","QQQ","IWM")):
    keep = [n for n in names if n in df.index]
    if not keep:
        return "N/A"
    sub = df.loc[keep, ["1D","5D","1M","YTD"]].applymap(lambda x: f"{x:.2%}")
    return sub.to_markdown()

summary = f"""# Daily Summary — {pd.Timestamp.today():%Y-%m-%d}

## Benchmarks
{fmt_returns(returns)}

## Vol
Latest VIX: {float(returns.loc['^VIX','1D']):.2%} (1D) if '^VIX' in returns.index else 'N/A'

## Latest Ideas
{(ideas.tail(5).to_markdown(index=False) if 'ideas' in globals() else 'N/A')}

## P&L (tail)
{(pnl.tail(5).to_markdown(index=False) if 'pnl' in globals() else 'N/A')}

"""

fname = f"daily_summary_{pd.Timestamp.today():%Y%m%d}.md"
with open(fname, "w") as f:
    f.write(summary)

print(f"Wrote {fname}")
print(summary)


## 11) (Optional) Save logs to Google Drive

In [ ]:
# Run this cell to mount Google Drive and save logs
# from google.colab import drive
# drive.mount('/content/drive')

# SAVE_DIR = "/content/drive/MyDrive/DailyQuantNotes"
# os.makedirs(SAVE_DIR, exist_ok=True)

# for name in ["ideas_log.csv","pnl_log.csv","risk_log.csv","daily_notes.md"]:
#     if os.path.exists(name):
#         !cp -f $name $SAVE_DIR/
# print("Synced logs to Drive.")

---
### 🧩 Notes
- Edit tables in-place, then run the save lines (commented) to persist.
- To add custom factors/signals, build on the **Technicals** section.
- Consider adding your own **pipeline** (universe, filters, alpha) block.
- For econ calendars, paste from your source or use APIs per your access.

**Happy trading & researching!**